In [ ]:
import tensorflow as tf
import dill
import contrastive
from contrastive import utils as contrastive_utils
from acme import specs
import jax
import numpy as np

import imageio
from base64 import b64encode
from IPython.display import display, HTML
import tempfile
import tqdm
import matplotlib.pyplot as plt
import jax.numpy as jnp
import gym
from acme.wrappers import gym_wrapper

import metaworld
# from contrastive_rl import classifier_envs
from acme.wrappers import gym_wrapper
from acme.wrappers import step_limit
# from contrastive_rl import classifier_utils
# from metaworld.envs.mujoco.utils import rotation

In [ ]:
# Wasn't able to use the checkpoints that Ben shared since there were some structures
# from Google's internal libaries. Kyle's checkpoints work though!
filename = './checkpoints/learner/'  # <-- Of course, this will be different for you
reader = tf.train.load_checkpoint(filename)
params = reader.get_tensor('learner/.ATTRIBUTES/py_state')
state = dill.loads(params)

In [ ]:
env, obs_dim = contrastive_utils.make_environment('fixed-goal-point_Cross', start_index=0, end_index=-1, seed=0)
spec = specs.make_environment_spec(env)
networks = contrastive.make_networks(spec, obs_dim,
                             actor_min_std=1e-6,  # <-- Make sure this is the same value used for training.
                             use_image_obs=False)

key = jax.random.PRNGKey(0)

# @jax.jit
def _act(obs, key):
  dist_params = networks.policy_network.apply(
    state.policy_params, obs[None])
  key, rng = jax.random.split(key, 2)
  # action = dist_params.mode()  # <-- Use this if you want to act deterministically
  action = networks.sample(dist_params, rng)
  return action[0], key

ts = env.reset()
_act(ts.observation, key)

In [ ]:
state.policy_params.keys()

In [ ]:
# the video writer commands will have to change here. I'd probably recommend using
# imageio. You just make a list of all the images, and then call imageio.mimsave(filename, image_list)

for _ in range(10):
  video_filename = '/tmp/video.mp4'
  writer = []

  vec = []
  ts = env.reset()
  env._goal_img = ts.observation.copy()
  for t in tqdm.trange(50):
    a, key = _act(ts.observation, key)
    ts = env.step(a.to_py())
    # use the following lines for image-based environments.
#     s = ts.observation[:obs_dim]
    g = ts.observation[obs_dim:]
    img = env.render(mode='rgb_array')
#     img = np.concatenate([
#       s.reshape((64, 64, 3)),
#       g.reshape((64, 64, 3))
#     ], axis=1)
    # for state-based environments, use the following
    # img = env.render(mode='rgb_array'), or something like that
    writer.append(img)
    if ts.last():
      break
  imageio.mimsave('output.gif', writer)

  mp4 = open('output.gif','rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  html = HTML("""
  <video width=800 framerate=1 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url)
  display(html)
# Not sure if the _dist variable applies to a state based environment? Commenting for now
  plt.plot(env.environment.environment.environment._dist)
  plt.show()